In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import warnings

In [10]:
books = pd.read_csv("data/clean/books_clean.csv")
ratings = pd.read_csv("data/clean/ratings_clean.csv")
users = pd.read_csv("data/clean/users_clean.csv")
books_users_ratings = pd.read_csv("data/clean/books_users_ratings.csv")

**Recommendations based on correlations**

In [11]:
#dropping books with one review to reduce size of dataset
average_ratings = pd.DataFrame(books_users_ratings.groupby('unique_isbn')['book_rating'].mean())
average_ratings['rating_count'] = pd.DataFrame(books_users_ratings.groupby('unique_isbn')['book_rating'].count())
average_ratings.sort_values('rating_count', ascending=False).head()
average_ratings['greater1'] = (average_ratings.rating_count > 9)

In [12]:
average_ratings.head()

,book_rating,rating_count,greater1
unique_isbn,,,
0000913154,8.0,1,False
000104687X,6.0,1,False
0001047213,9.0,1,False
000104799X,7.5,2,False
0001061127,8.0,1,False


In [13]:
books_users_ratings['dropornot'] = [average_ratings.greater1[x] == False for x in books_users_ratings.unique_isbn]

In [14]:
reduced_books_users_ratings = books_users_ratings[books_users_ratings.dropornot == False]

In [17]:
# dropping users with one review
userdrop = reduced_books_users_ratings.groupby('user_id')['book_rating'].count() <= 2
reduced_books_users_ratings['dropuser'] = [userdrop[x] for x in reduced_books_users_ratings['user_id']]

/Users/joshuagoldberg/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [25]:
reduced_books_users_ratings = pd.merge(reduced_books_users_ratings, users, on="user_id")
reduced_books_users_ratings.head()

,user_id,isbn,book_rating,book_title,book_author,year_of_publication,unique_isbn,dropornot,dropuser,age,city,state,country
0,276744,038550120X,7,a painted house,john grisham,2001,044023722X,False,True,34.786876,torrance,california,usa
1,11676,038550120X,10,a painted house,john grisham,2001,044023722X,False,False,34.786876,NaN,n/a,n/a
2,11676,044023722X,8,a painted house,john grisham,2001,044023722X,False,False,34.786876,NaN,n/a,n/a
3,11676,0671537458,8,waiting to exhale,terry mcmillan,1992,0671537458,False,False,34.786876,NaN,n/a,n/a
4,11676,0671864173,5,waiting to exhale,terry mcmillan,1993,0671537458,False,False,34.786876,NaN,n/a,n/a


In [26]:
reduced_books_users_ratings = reduced_books_users_ratings[reduced_books_users_ratings.dropuser == False]

In [27]:
reduced_books_users_ratings.to_csv("data/clean/reduced_books_users_ratings.csv", index=False)

In [67]:
user_item_rating = reduced_books_users_ratings[['user_id', 'unique_isbn', 'book_rating']]
user_item_rating.head()

,user_id,unique_isbn,book_rating
4,11676,038550120X,10
5,11676,0671537458,8
6,11676,0679776818,8
7,11676,0684867621,3
9,11676,8437606322,8


In [68]:
#dividing into test and train
from sklearn import model_selection
train_data, test_data = model_selection.train_test_split(user_item_rating, test_size=0.40)

In [69]:
train_users= len(train_data['user_id'].unique())
train_books =len(train_data['unique_isbn'].unique()) 

In [171]:
train_users

10851

In [172]:
train_books

5572

In [70]:
test_users = len(test_data['user_id'].unique())
test_books = len(test_data['unique_isbn'].unique())

In [173]:
test_users

10085

In [174]:
test_books

5519

In [71]:
#creating a user-item matrix to calculate similarity between users


In [72]:
train_matrix = np.zeros((train_users,train_books))

In [73]:
#Map the user_id and unique_isbn fields to sequential integers for matrix processing
u_unique_train = train_data.user_id.unique()  # create a 'set' (i.e. all unique) list of vals
train_data_user2idx = {o:i for i, o in enumerate(u_unique_train)}
# Get int mapping for unique_isbn
b_unique_train = train_data.unique_isbn.unique()  # create a 'set' (i.e. all unique) list of vals
train_data_book2idx= {o:i for i, o in enumerate(b_unique_train)}

### TESTING SET
# Get int mapping for user_id
u_unique_test = test_data.user_id.unique()  # create a 'set' (i.e. all unique) list of vals
test_data_user2idx = {o:i for i, o in enumerate(u_unique_test)}
# Get int mapping for unique_isbn
b_unique_test = test_data.unique_isbn.unique()  # create a 'set' (i.e. all unique) list of vals
test_data_book2idx = {o:i for i, o in enumerate(b_unique_test)}

In [77]:
train_data['u_unique'] = train_data['user_id'].map(train_data_user2idx)
train_data['b_unique'] = train_data['unique_isbn'].map(train_data_book2idx)

### TESTING SET
test_data['u_unique'] = test_data['user_id'].map(test_data_user2idx)
test_data['b_unique'] = test_data['unique_isbn'].map(test_data_book2idx)



C:\Users\Ami\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\Ami\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Ami\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata

In [78]:
### Convert back to 3-column df
train_data_new = train_data[['u_unique', 'b_unique', 'book_rating']]
test_data_new = test_data[['u_unique', 'b_unique', 'book_rating']]

In [79]:
for entry in train_data_new.itertuples(): # entry[1] is the user-id, entry[2] is the book-isbn
    train_matrix[entry[1]-1, entry[2]-1] = entry[3] # -1 is to counter 0-based indexing

In [80]:
train_matrix

array([[10.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0., 10.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0., 10., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  8.]])

In [109]:
#Calculating cosine similarity
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(train_matrix, metric='cosine')
item_similarity = pairwise_distances(train_matrix.T, metric='cosine') # .T transposes the matrix (NumPy)

In [110]:
def predict(ratings, similarity, type='user'): # default type is 'user'
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        # Use np.newaxis so that mean_user_rating has the same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [111]:
#predicts ratings for every book and every user
item_prediction = predict(train_matrix, item_similarity, type='item')
user_prediction = predict(train_matrix, user_similarity, type='user')

In [267]:
#function to get top ten recommendations for a given user id based on user similarity scores
def predict_user(user_id, user_prediction, books_users_ratings):
    uniqueuser = train_data_user2idx[user_id]
    ranks = pd.DataFrame(user_prediction[uniqueuser])
    ranks['uniquebook'] = [i for i,o in enumerate(ranks[0])]
    ranks.columns = ['rating', 'uniquebook']
    ranks= ranks.sort_values(by=['rating'],ascending=False)
    isbn = []
    for i,v in enumerate(ranks.uniquebook):
        for key, value in train_data_book2idx.items():
            if v == value:
                isbn.append(key)
    ranks['unique_isbn'] = isbn
    ranks['rated'] = [1 if x in reduced_books_users_ratings[reduced_books_users_ratings.user_id == user_id].unique_isbn.tolist() else 0 for x in ranks.unique_isbn]
    ranks = ranks[ranks.rated==0]
    ranks=ranks.head(10)
    recommendations = [books_users_ratings[books_users_ratings.unique_isbn==x].book_title.tolist()[1] for x in ranks.unique_isbn]
    ranks['titles'] =recommendations
    print(ranks)

In [268]:
#example prediction of ten books for a given user based on user similarity scores
predict_user(161882,user_prediction,books_users_ratings)

       rating  uniquebook unique_isbn  rated  \
365  0.241631         365  0553569155      0   
539  0.195143         539  0440236819      0   
248  0.171858         248  0553569031      0   
43   0.156430          43  0060977493      0   
211  0.146043         211  0688177751      0   
738  0.141077         738  0380725835      0   
11   0.139725          11  0451525795      0   
306  0.137516         306  0515116750      0   
552  0.137500         552  0060002050      0   
407  0.129629         407  0811813401      0   

                                                titles  
365                                       Seinlanguage  
539                                        The Cottage  
248                             Gone But Not Forgotten  
43                             The God of Small Things  
211               Miss Julia Speaks Her Mind : A Novel  
738  Endangered Species (Anna Pigeon Mysteries (Pap...  
11                         My Antonia (Signet Classic)  
306            

In [269]:
#function to get top ten recommendations for a given user id based on item similarity scores
def predict_user(user_id, item_prediction, books_users_ratings):
    uniqueuser = train_data_user2idx[user_id]
    ranks = pd.DataFrame(item_prediction[uniqueuser])
    ranks['uniquebook'] = [i for i,o in enumerate(ranks[0])]
    ranks.columns = ['rating', 'uniquebook']
    ranks= ranks.sort_values(by=['rating'],ascending=False)
    isbn = []
    for i,v in enumerate(ranks.uniquebook):
        for key, value in train_data_book2idx.items():
            if v == value:
                isbn.append(key)
    ranks['unique_isbn'] = isbn
    ranks['rated'] = [1 if x in reduced_books_users_ratings[reduced_books_users_ratings.user_id == user_id].unique_isbn.tolist() else 0 for x in ranks.unique_isbn]
    ranks = ranks[ranks.rated==0]
    ranks=ranks.head(10)
    recommendations = [books_users_ratings[books_users_ratings.unique_isbn==x].book_title.tolist()[1] for x in ranks.unique_isbn]
    ranks['titles'] =recommendations
    print(ranks)

In [271]:
#example prediction of ten books for a given user based on item similarity scores
predict_user(161882,item_prediction,books_users_ratings)

        rating  uniquebook unique_isbn  rated  \
4784  0.037389        4784  0062554468      0   
906   0.037325         906  0689817851      0   
2887  0.037310        2887  0060975547      0   
620   0.037305         620  0142002267      0   
4266  0.037302        4266  0312852487      0   
5275  0.037290        5275  087542127X      0   
2621  0.037276        2621  0446530077      0   
4057  0.037256        4057  0743467191      0   
1782  0.037247        1782  0553211757      0   
5456  0.037246        5456  0440501601      0   

                                                 titles  
4784  Codependent No More: How to Stop Controlling O...  
906                                        Go Ask Alice  
2887                                      Love Medicine  
620   Word Freak: Heartbreak, Triumph, Genius, and O...  
4266      The Dragon Reborn (The Wheel of Time, Book 3)  
5275  The Truth About Witchcraft Today (Llewellyn's ...  
2621                                            Find M

In [275]:
### Testing the prediction methods bases on rmse
# Create user-item matrices
n_users = test_data['u_unique'].nunique()
n_books = test_data['b_unique'].nunique()

# First, create an empty matrix of size USERS x BOOKS (this speeds up the later steps)
test_matrix = np.zeros((n_users, n_books))


In [276]:
test_data.head()

,user_id,unique_isbn,book_rating,u_unique,b_unique
97905,227836,0515131229,8,0,0
155656,232052,034540288X,10,1,1
271929,154788,0689711522,9,2,2
50917,182053,0786817070,10,3,3
69251,243720,002542730X,5,4,4


In [277]:

# Then, add the appropriate vals to the matrix by extracting them from the df with itertuples
for entry in test_data_new.itertuples(): # entry[1] is the user-id, entry[2] is the book-isbn
    test_matrix[entry[1]-1, entry[2]-1] = entry[3] # -1 is to counter 0-based indexing

In [278]:
from sklearn.metrics import mean_squared_error
from math import sqrt

def rmse(prediction, test_matrix):
    prediction = prediction[test_matrix.nonzero()].flatten()
    test_matrix = test_matrix[test_matrix.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, test_matrix))

# Call on test set to get error from each approach ('user' or 'item')
print(f'User-based CF RMSE: {rmse(user_prediction, test_matrix)}')
print(f'Item-based CF RMSE: {rmse(item_prediction, test_matrix)}')

User-based CF RMSE: 7.973753170432789
Item-based CF RMSE: 7.976354848069027
